In [27]:
# Import the modules
import pandas as pd
from pathlib import Path
import hvplot.pandas

# Import the K-means algorithm
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [67]:
# Read in the CSV file as a Pandas Dataframe
df = pd.read_csv(Path("resources/baseballData.csv"))


In [68]:

df.set_index("last_name, first_name", inplace=True)
df.drop(columns=["player_id"], inplace=True)
df.head()

,year,player_age,ab,pa,hit,single,double,triple,home_run,strikeout,...,batted_ball,f_strike_percent,groundballs_percent,groundballs,flyballs_percent,flyballs,linedrives_percent,linedrives,popups_percent,popups
"last_name, first_name",,,,,,,,,,,,,,,,,,,,,
"Hunter, Torii",2015,39,521,567,125,81,22,0,22,105,...,421,64.0,49.4,208,22.6,95,20.9,88,7.1,30
"Ortiz, David",2015,39,528,614,144,70,37,0,37,95,...,442,52.1,37.6,166,25.6,113,28.7,127,8.1,36
"Rodriguez, Alex",2015,39,523,620,131,75,22,1,33,145,...,385,60.3,43.6,168,24.9,96,24.9,96,6.5,25
"Ramirez, Aramis",2015,37,475,516,117,68,31,1,17,68,...,412,65.5,37.6,155,24.5,101,29.1,120,8.7,36
"Beltré, Adrián",2015,36,567,619,163,109,32,4,18,65,...,510,61.9,42.4,216,18.4,94,31.6,161,7.6,39


In [52]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# Standard scale helps normalize the data by replacing N/A values with true values, helps for morre prediction, for instance if some values for percentage are not divided by 100 then it can help remedy that
#1. Fit Scaler into dataframe
scaler = StandardScaler()
scaler.fit(df)

StandardScaler()

In [53]:
# Create a list with the number of k-values from 1 to 11
# Create a DataFrame with the scaled data
normal = scaler.transform(df)
# Create an empty list to store the inertia values
kValue = list(range(1, 12))

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
inertia = []
for x in kValue:
    kmeans = KMeans(n_clusters=x, random_state=0)
    kmeans.fit(df)
    inertia.append(kmeans.inertia_)
print(inertia)


[460299649.4490651, 75550308.93887225, 45328839.49646305, 39441947.12875338, 32911722.744241804, 29803796.717375495, 27690379.74969953, 25798861.134715777, 24522386.055136334, 23639447.19254012, 23009959.283346497]


In [39]:
# Create a dictionary with the data to plot the Elbow curve
elbowDict = {'k' : kValue, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbowDf = pd.DataFrame(elbowDict)
elbowDf

,k,inertia
0,1,1.072830e+13
1,2,3.758960e+12
2,3,2.320160e+12
3,4,1.051154e+12
4,5,4.473020e+11
5,6,3.629845e+11
6,7,2.703447e+11
7,8,2.097112e+11
8,9,1.585659e+11
9,10,1.398180e+11


In [40]:
originalElbow = elbowDf.hvplot.line(
    x='k',
    y='inertia',
    xlabel='k-values',
    ylabel='inertia',
    title='inertia for each k-value',
)
originalElbow

:Curve   [k]   (inertia)

In [54]:
# Create and initialize the K-means model instance for 3 clusters
model = KMeans(n_clusters=3, random_state=1)

# Print the model
model
# Fit the data to the instance of the model
model.fit(df)
# Make predictions about the data clusters using the trained model
customer_segment_3 = model.predict(df)

# Print the predictions
print(customer_segment_3)

[2 0 0 ... 1 1 1]


In [55]:
predictDF = df
predictDF['pred_cluster'] = customer_segment_3

In [69]:
cryptoCluster = predictDF.hvplot.scatter(
    x='on_base_percent',
    y='slg_percent',
    by='pred_cluster',
    hover_cols='last_name, first_name',
    xlabel='On Base Percent',
    ylabel='Slugging',
    alpha=0.5
)
cryptoCluster

:NdOverlay   [pred_cluster]
   :Scatter   [on_base_percent]   (slg_percent,last_name, first_name)